In [2]:
import numpy as np
import matplotlib as plt
import pymysql
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import pandas as pd
% matplotlib inline

## 8/2

In [5]:
# vips 공시 database에서 유가증권, 코스피 공시 데이터만 select >> dataframe으로 return
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='dg',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from dg.searchDB where (crp_cls = 'K' or crp_cls = 'Y')"
    curs.execute(sql)
    rcp_table = curs.fetchall()

server.stop()

In [7]:
with SSHTunnelForwarder(
         ('1.212.234.245', 486),
         ssh_password="vips",
         ssh_username="vips",
         remote_bind_address=('127.0.0.1', 3306)) as server:

    conn = pymysql.connect(host='localhost',
                           port=server.local_bind_port,
                           user='root',
                           passwd='qlqtmajrwk',
                           db='dg',
                           charset='utf8')

    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql2 = "select * from dg.stockDB"
    curs.execute(sql2)
    stock_table = curs.fetchall()
    conn.commit()
    conn.close()
    
    
server.stop()

In [5]:
# 0812 13:07 기준 1297983개 공시
len(rcp_table)

1297983

In [8]:
# convert from list of dict  to dataframe
rcp_df = pd.DataFrame.from_dict(rcp_table)

In [9]:
rcp_df['rcp_dt'] = pd.to_datetime(rcp_df['rcp_dt'])
rcp_df['rcp_year'] = rcp_df['rcp_dt'].dt.year
rcp_df['rcp_month'] = rcp_df['rcp_dt'].dt.month
rcp_df['rcp_day'] = rcp_df['rcp_dt'].dt.day

In [205]:
rcp_df['rcp_dayofweek'] = rcp_df['rcp_dt'].dt.dayofweek
# The day of the week with Monday=0, Sunday=6

In [10]:
rcp_df.tail()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day
1298242,189980,K,흥국에프엔비,흥국에프엔비,2018-08-03,20180803900262,14:59,코,None,연결재무제표기준영업(잠정)실적(공정공시),2018,8,3
1298243,036930,K,주성엔지니어링,주성엔지니어링,2018-08-03,20180803900279,15:17,코,None,연결재무제표기준영업(잠정)실적(공정공시),2018,8,3
1298244,064760,K,티씨케이,티씨케이,2018-08-03,20180803900284,15:12,코,None,[기재정정]주식명의개서정지(주주명부폐쇄),2018,8,3
1298245,069540,K,라이트론,라이트론,2018-08-03,20180803900292,15:35,코,None,증권발행결과(자율공시)(제3자배정 유상증자),2018,8,3
1298246,043610,K,지니뮤직,지니뮤직,2018-08-03,20180803900295,15:36,코,None,영업(잠정)실적(공정공시),2018,8,3


In [25]:
rcp_df.groupby('rcp_day')['rcp_no'].count()

rcp_day
1     31749
2     41887
3     37127
4     38264
5     35365
6     36216
7     40725
8     43111
9     42213
10    50875
11    38990
12    42138
13    47742
14    68622
15    53263
16    46661
17    35189
18    34623
19    33849
20    36395
21    36344
22    37300
23    37213
24    37979
25    37535
26    40323
27    45300
28    48797
29    48078
30    54580
31    39517
Name: rcp_no, dtype: int64

In [15]:
rcp_df.groupby('rcp_dayofweek')['rcp_no'].count()

rcp_dayofweek
0    242979
1    236425
2    224200
3    256788
4    318061
5     19530
Name: rcp_no, dtype: int64

In [16]:
rcp_df.groupby('rcp_year')['rcp_no'].count()

rcp_year
1999      1102
2000     20387
2001     32075
2002     45489
2003     61364
2004     63287
2005     64288
2006     61674
2007     68520
2008     70355
2009     79657
2010     74374
2011     77785
2012     75321
2013     80175
2014     84694
2015     91806
2016     92725
2017    100678
2018     52227
Name: rcp_no, dtype: int64

## rcp_dt를 언제부터 쌓고 있는지 확인

In [20]:
rcp_df[lambda x : x.rcp_year == 2007] #68520

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day,rcp_dayofweek
335053,011780,Y,금호석유화학,금호석유화학,2007-06-05,20060918800050,None,유,ET_01,기타경영사항(자율공시),2007,6,5,1
345297,034120,Y,SBS,SBS,2007-02-09,20061201800222,None,유,None,주주총회소집결의,2007,2,9,4
345811,000680,Y,LS네트웍스,LS네트웍스,2007-01-09,20061206800176,None,유,G_02,최대주주등소유주식변동신고서(최대주주변경시),2007,1,9,1
346428,008040,Y,사조동아원,사조동아원,2007-01-09,20061211800103,None,유,G_02,최대주주등소유주식변동신고서(최대주주변경시),2007,1,9,1
346656,002380,Y,케이씨씨,케이씨씨,2007-01-08,20061212800243,None,유,E_4,결산실적공시예고(안내공시),2007,1,8,0
346869,005930,Y,삼성전자,삼성전자,2007-01-15,20061213800072,None,유,G_01,최대주주등소유주식변동신고서,2007,1,15,0
347190,003570,Y,S&T중공업,S&T중공업,2007-03-13,20061214800331,None,유,B_7,단일판매ㆍ공급계약체결,2007,3,13,1
348130,002380,Y,케이씨씨,케이씨씨,2007-02-23,20061219800062,None,유,None,정기주주총회결과,2007,2,23,4
348271,007610,Y,선도전기,선도전기,2007-02-16,20061220800013,None,유,None,주주총회소집결의,2007,2,16,4
348480,006740,Y,영풍제지,영풍제지,2007-01-02,20061221800014,None,유,B_23,금전대여결정(자율공시),2007,1,2,1


In [93]:
rcp_df[rcp_df['rcp_rt'].isnull()]

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day,rcp_dayofweek
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12),1999,3,25,3
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12),1999,3,25,3
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12),1999,3,25,3
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12),1999,3,25,3
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12),1999,3,25,3
5,003580,Y,동원,동원,1999-03-26,19990326000007,None,,E_1,사업보고서 (1998.12),1999,3,26,4
6,005030,Y,부산주공,부산주공,1999-03-27,19990327000001,None,,E_1,사업보고서 (1998.12),1999,3,27,5
7,014910,Y,성문전자,성문전자,1999-03-27,19990327000005,None,정,E_1,사업보고서 (1998.12),1999,3,27,5
8,029530,Y,신도리코,신도리코,1999-03-29,19990329000001,None,정,E_1,사업보고서 (1998.12),1999,3,29,0
9,005320,Y,국동,국동,1999-03-29,19990329000003,None,정연,E_1,사업보고서 (1998.12),1999,3,29,0


In [176]:
# 2016년 7월 31일까지 9:00~15:00 / 2016년 8월 1일부터 9:00~15:30
rcp_df.groupby(['rcp_no', 'rcp_hour']).count()

,,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_rt,rmk,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day,rcp_dayofweek
rcp_no,rcp_hour,,,,,,,,,,,,,
19990325000001,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990325000003,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990325000008,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990325000015,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990325000016,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990326000007,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990327000001,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990327000005,0,1,1,1,1,1,0,1,1,1,1,1,1,1
19990329000001,0,1,1,1,1,1,0,1,1,1,1,1,1,1


## rcp_df에 datecd 만들기 

In [11]:
rcp_df['rcp_year'] = rcp_df['rcp_year'].astype(str)
rcp_df['rcp_month'] = rcp_df['rcp_month'].astype(str)
rcp_df['rcp_day'] = rcp_df['rcp_day'].astype(str)
rcp_df['crp_cd'] = rcp_df['crp_cd'].astype(str)

In [12]:
rcp_df['rcp_month'] = rcp_df['rcp_month'].str.zfill(2)
rcp_df['rcp_day'] = rcp_df['rcp_day'].str.zfill(2)

In [13]:
rcp_df['datecd'] = rcp_df.loc[:,'rcp_year'] + rcp_df.loc[:,'rcp_month']\
+ rcp_df.loc[:,'rcp_day'] + rcp_df.loc[:,'crp_cd']

In [91]:
print(rcp_df.iloc[1255956])
print(rcp_df.iloc[1255957]) #1255957 부터 시간 있음 2018-02-12 13:43

crp_cd                        095190
crp_cls                            K
crp_nm                         이엠코리아
flr_nm                         이엠코리아
rcp_dt           2018-02-12 00:00:00
rcp_no                20180212900276
rcp_rt                          None
rmk                                코
rpt_cat                          B_9
rpt_nm             단일판매ㆍ공급계약체결(자율공시)
rcp_year                        2018
rcp_month                          2
rcp_day                           12
rcp_dayofweek                      0
Name: 1255956, dtype: object
crp_cd                        086670
crp_cls                            K
crp_nm                           비엠티
flr_nm                           비엠티
rcp_dt           2018-02-12 00:00:00
rcp_no                20180212900282
rcp_rt                         13:43
rmk                                코
rpt_cat                          E_4
rpt_nm                영업(잠정)실적(공정공시)
rcp_year                        2018
rcp_month                          2
rcp_day  

## stock_df 와 join (key = datecd)

In [14]:
rcp_df.tail()

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day,datecd
1298242,189980,K,흥국에프엔비,흥국에프엔비,2018-08-03,20180803900262,14:59,코,None,연결재무제표기준영업(잠정)실적(공정공시),2018,08,03,20180803189980
1298243,036930,K,주성엔지니어링,주성엔지니어링,2018-08-03,20180803900279,15:17,코,None,연결재무제표기준영업(잠정)실적(공정공시),2018,08,03,20180803036930
1298244,064760,K,티씨케이,티씨케이,2018-08-03,20180803900284,15:12,코,None,[기재정정]주식명의개서정지(주주명부폐쇄),2018,08,03,20180803064760
1298245,069540,K,라이트론,라이트론,2018-08-03,20180803900292,15:35,코,None,증권발행결과(자율공시)(제3자배정 유상증자),2018,08,03,20180803069540
1298246,043610,K,지니뮤직,지니뮤직,2018-08-03,20180803900295,15:36,코,None,영업(잠정)실적(공정공시),2018,08,03,20180803043610


In [15]:
# convert from list of dict  to dataframe
stock_df = pd.DataFrame.from_dict(stock_table)

In [16]:
len(stock_table)

6818450

stock_df['diff'] = stock_df['diff'].astype(int)
stock_df['high'] = stock_df['high'].astype(int)
stock_df['low'] = stock_df['low'].astype(int)
stock_df['open'] = stock_df['open'].astype(int)
stock_df['volume'] = stock_df['volume'].astype(int)

In [17]:
stock_df.head()

,close,crp_cd,date,datecd,diff,high,low,open,volume
0,36000.0,000020,1996-06-25,19960625000020,0.0,38400.0,36000.0,37700.0,102200.0
1,9300.0,000040,1996-06-25,19960625000040,0.0,9390.0,9290.0,9390.0,1600.0
2,42000.0,000050,1996-06-25,19960625000050,0.0,42000.0,42000.0,42000.0,0.0
3,30500.0,000060,1996-06-25,19960625000060,0.0,31000.0,29700.0,30100.0,68220.0
4,24700.0,000070,1996-06-25,19960625000070,0.0,25600.0,24700.0,25500.0,16960.0


In [18]:
join_df = rcp_df.merge(stock_df, on = 'datecd', how = 'left') #join과 merge의 차이점?

In [19]:
join_df.head()

,crp_cd_x,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,...,rcp_day,datecd,close,crp_cd_y,date,diff,high,low,open,volume
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12),...,25,19990325023150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12),...,25,19990325001140,7800.0,001140,1999-03-25,40.0,8000.0,7620.0,7800.0,10080.0
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12),...,25,19990325014160,10500.0,014160,1999-03-25,150.0,11700.0,10500.0,11500.0,454110.0
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12),...,25,19990325002310,10800.0,002310,1999-03-25,300.0,11200.0,10500.0,10550.0,75860.0
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12),...,25,19990325015360,25400.0,015360,1999-03-25,0.0,25600.0,24900.0,24900.0,6900.0


In [24]:
import pickle

In [43]:
f = open('join_df.bin', 'wb')

In [44]:
pickle.dump(join_df, f)

In [45]:
f.close()

In [46]:
data = pickle.load(open('join_df.bin', 'rb'))

In [47]:
data.head()

,crp_cd_x,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,...,datecd,close,crp_cd_y,date,diff,high,low,open,volume,setting_y
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12),...,19990325023150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12),...,19990325001140,7800.0,001140,1999-03-25,40.0,8000.0,7620.0,7800.0,10080.0,1
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12),...,19990325014160,10500.0,014160,1999-03-25,150.0,11700.0,10500.0,11500.0,454110.0,0
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12),...,19990325002310,10800.0,002310,1999-03-25,300.0,11200.0,10500.0,10550.0,75860.0,1
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12),...,19990325015360,25400.0,015360,1999-03-25,0.0,25600.0,24900.0,24900.0,6900.0,1


## 당일종가 - 당일시가

In [20]:
join_df['setting_y'] = [1 if x >= 0 else 0 for x in join_df['close'] - join_df['open']]

In [48]:
join_df.head()

,crp_cd_x,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,...,datecd,close,crp_cd_y,date,diff,high,low,open,volume,setting_y
0,023150,Y,MH에탄올,MH에탄올,1999-03-25,19990325000001,None,정,E_1,사업보고서 (1998.12),...,19990325023150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,001140,Y,국보,국보,1999-03-25,19990325000003,None,정,E_1,사업보고서 (1998.12),...,19990325001140,7800.0,001140,1999-03-25,40.0,8000.0,7620.0,7800.0,10080.0,1
2,014160,Y,대영포장,대영포장,1999-03-25,19990325000008,None,정연,E_1,사업보고서 (1998.12),...,19990325014160,10500.0,014160,1999-03-25,150.0,11700.0,10500.0,11500.0,454110.0,0
3,002310,Y,아세아제지,아세아제지,1999-03-25,19990325000015,None,정,E_1,사업보고서 (1998.12),...,19990325002310,10800.0,002310,1999-03-25,300.0,11200.0,10500.0,10550.0,75860.0,1
4,015360,Y,예스코,예스코,1999-03-25,19990325000016,None,정,E_1,사업보고서 (1998.12),...,19990325015360,25400.0,015360,1999-03-25,0.0,25600.0,24900.0,24900.0,6900.0,1


In [58]:
join_df.columns

Index(['crp_cd_x', 'crp_cls', 'crp_nm', 'flr_nm', 'rcp_dt', 'rcp_no', 'rcp_rt',
       'rmk', 'rpt_cat', 'rpt_nm', 'rcp_year', 'rcp_month', 'rcp_day',
       'datecd', 'close', 'crp_cd_y', 'date', 'diff', 'high', 'low', 'open',
       'volume', 'setting_y'],
      dtype='object')

In [61]:
pickle_df = join_df[['crp_cd_x', 'crp_cls', 'crp_nm', 'rcp_dt', 'rcp_no', 'rpt_cat', 'rpt_nm',
                     'rcp_year', 'rcp_month', 'rcp_day', 'datecd', 'close', 'open', 'high', 'low',
                     'open', 'volume', 'setting_y']]

In [74]:
pickle_df

,crp_cd_x,crp_cls,crp_nm,rcp_dt,rcp_no,rpt_cat,rpt_nm,rcp_year,rcp_month,rcp_day,datecd,close,open,diff,high,low,open,volume,setting_y
0,023150,Y,MH에탄올,1999-03-25,19990325000001,E_1,사업보고서 (1998.12),1999,03,25,19990325023150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,001140,Y,국보,1999-03-25,19990325000003,E_1,사업보고서 (1998.12),1999,03,25,19990325001140,7800.0,7800.0,40.0,8000.0,7620.0,7800.0,10080.0,1
2,014160,Y,대영포장,1999-03-25,19990325000008,E_1,사업보고서 (1998.12),1999,03,25,19990325014160,10500.0,11500.0,150.0,11700.0,10500.0,11500.0,454110.0,0
3,002310,Y,아세아제지,1999-03-25,19990325000015,E_1,사업보고서 (1998.12),1999,03,25,19990325002310,10800.0,10550.0,300.0,11200.0,10500.0,10550.0,75860.0,1
4,015360,Y,예스코,1999-03-25,19990325000016,E_1,사업보고서 (1998.12),1999,03,25,19990325015360,25400.0,24900.0,0.0,25600.0,24900.0,24900.0,6900.0,1
5,003580,Y,동원,1999-03-26,19990326000007,E_1,사업보고서 (1998.12),1999,03,26,19990326003580,63400.0,63200.0,700.0,64700.0,63200.0,63200.0,29170.0,1
6,005030,Y,부산주공,1999-03-27,19990327000001,E_1,사업보고서 (1998.12),1999,03,27,19990327005030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,014910,Y,성문전자,1999-03-27,19990327000005,E_1,사업보고서 (1998.12),1999,03,27,19990327014910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,029530,Y,신도리코,1999-03-29,19990329000001,E_1,사업보고서 (1998.12),1999,03,29,19990329029530,44500.0,45000.0,1500.0,45000.0,44000.0,45000.0,10760.0,0
9,005320,Y,국동,1999-03-29,19990329000003,E_1,사업보고서 (1998.12),1999,03,29,19990329005320,21200.0,21900.0,900.0,21900.0,21100.0,21900.0,11100.0,0


In [76]:
pickle_df['crp_cls'].unique()

array(['Y', 'K'], dtype=object)

In [71]:
f = open('df_for_ml.bin', 'wb')

In [72]:
pickle.dump(pickle_df, f)

In [73]:
f.close()

In [56]:
join_df.rpt_cat.unique()

array(['E_1', 'E_3', 'GS', 'G_06', 'G_08', None, 'L1_03', 'T_1', 'L1_04',
       'I_07', 'B_25', 'B_16', 'K_3', 'B_7', 'B_3', 'I_09', 'K_2', 'I_01',
       'I_03', 'G_10', 'B_2', 'E_4', 'I_08', 'B_4', 'K_4', 'B_1', 'B_23',
       'T_2', 'T_5', 'K_5', 'K_1', 'E_2', 'I_12', 'MS_04', 'B_5', 'I_10',
       'I_17', 'B_6', 'I_06', 'B_8', 'T_3', 'K_7', 'T_4', 'H2_09', 'G_09',
       'G_04', 'R_01', 'ET_01', 'N_01', 'N_03', 'R_02', 'R_03', 'B_21',
       'B_9', 'B_10', 'G_01', 'G_02', 'B_24', 'B_22', 'L1_01', 'S_03',
       'S_02', 'K_6', 'G_03', 'MS_01', 'H2_10', 'H2_13', 'C_03', 'H2_11',
       'G_07', 'H2_03', 'H2_04', 'L1_02', 'C_01', 'H2_01', 'N_05', 'N_04',
       'C_02', 'S_01', 'L2_04', 'L2_03', 'L2_05', 'B_13', 'G_11', 'H2_12',
       'I_02', 'I_05', 'I_14', 'I_04', 'H2_02', 'I_15', 'I_16', 'H1_02',
       'H1_09', 'H1_03', 'H1_05', 'H1_06', 'H1_08', 'H1_04', 'H1_07',
       'H2_07', 'L2_06', 'R_04', 'R_05', 'H2_06', 'B_14', 'B_12', 'B_15',
       'B_11', 'N_02', 'B_20', 'H2_05', 'I_2

In [55]:
join_df.groupby('rpt_cat').size()

rpt_cat
B-03         1
B-07         1
B_1        420
B_10       982
B_11       241
B_12       150
B_13        87
B_14       214
B_15       363
B_16      4452
B_2        708
B_20       635
B_21        45
B_22     13599
B_23      1395
B_24       305
B_25      3349
B_3      10537
B_4        547
B_5        744
B_6        145
B_7      17150
B_8        374
B_9      14062
C_01       137
C_02        87
C_03       256
E-2          1
ET_01    14305
E_1      32311
         ...  
K_6        212
K_7        269
L1_01     2483
L1_02      410
L1_03    22273
L1_04    12634
L2_03     3319
L2_04     1595
L2_05      200
L2_06      145
MS_01     2615
MS_04     3109
N_01      2086
N_02       100
N_03      1106
N_04       169
N_05       120
R_01      8623
R_02      4801
R_03       992
R_04       134
R_05       198
S_01       395
S_02      2100
S_03      1588
T_1      12130
T_2       3803
T_3       1726
T_4       1958
T_5      14176
Length: 117, dtype: int64

In [23]:
join_df.groupby(['rpt_cat'])[lambda x : x not 'E'].size().sort_values(ascending = False)

TypeError: 'in <string>' requires string as left operand, not Series

## 당일종가 - 전일종가

In [75]:
#diff test
join_df.tail(10000)

,crp_cd_x,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rcp_rt,rmk,rpt_cat,rpt_nm,...,datecd,close,crp_cd_y,date,diff,high,low,open,volume,setting_y
1288247,005930,Y,삼성전자,이춘재,2018-06-12,20180612000243,15:30,,None,임원ㆍ주요주주특정증권등소유상황보고서,...,20180612005930,49400.0,005930,2018-06-12,500.0,49800.0,49250.0,49700.0,12110600.0,0
1288248,016360,Y,삼성증권,삼성증권,2018-06-12,20180612000246,15:31,,None,증권발행실적보고서,...,20180612016360,38150.0,016360,2018-06-12,50.0,38300.0,37700.0,38000.0,373937.0,1
1288249,272550,Y,삼양패키징,삼양패키징,2018-06-12,20180612000249,15:32,,None,[기재정정]사업보고서 (2017.12),...,20180612272550,17500.0,272550,2018-06-12,500.0,18000.0,17450.0,17900.0,19817.0,0
1288250,023450,Y,동남합성,동남합성,2018-06-12,20180612000250,15:36,,None,주요사항보고서(자기주식처분결정),...,20180612023450,61300.0,023450,2018-06-12,1000.0,61800.0,60000.0,60000.0,189.0,1
1288251,003470,Y,유안타증권,유안타증권,2018-06-12,20180612000254,15:33,,None,일괄신고추가서류(파생결합증권-주가연계증권),...,20180612003470,4595.0,003470,2018-06-12,10.0,4685.0,4540.0,4630.0,514692.0,0
1288252,005940,Y,NH투자증권,NH투자증권,2018-06-12,20180612000256,15:33,,None,투자설명서(일괄신고),...,20180612005940,16850.0,005940,2018-06-12,0.0,17100.0,16700.0,16800.0,848561.0,1
1288253,033340,K,좋은사람들,좋은사람들,2018-06-12,20180612000259,15:39,,None,주요사항보고서(자기주식처분결정),...,20180612033340,6770.0,033340,2018-06-12,260.0,7430.0,6000.0,7050.0,17676500.0,0
1288254,003540,Y,대신증권,대신증권,2018-06-12,20180612000262,15:37,,None,일괄신고추가서류(기타파생결합증권),...,20180612003540,13450.0,003540,2018-06-12,0.0,13550.0,13300.0,13450.0,75794.0,1
1288255,012030,Y,DB,박현철,2018-06-12,20180612000263,15:38,,None,임원ㆍ주요주주특정증권등소유상황보고서,...,20180612012030,1130.0,012030,2018-06-12,30.0,1170.0,1115.0,1160.0,1556100.0,0
1288256,005070,Y,코스모신소재,코스모신소재,2018-06-12,20180612000265,15:39,,None,[기재정정]사업보고서 (2017.12),...,20180612005070,22450.0,005070,2018-06-12,350.0,23250.0,21750.0,22500.0,584059.0,0
